# Práctica sobre remuestreo de los datos de la NASA 

Prompt  

Tengo datos meteorológicos de frecuencia: diaria, con atributo fecha en formato datetime de Python y quiero remuestrearlos con una función de usuario de Python, a una frecuencia semanal (Pero de semana epidemiológica de domingo a sábado) y a la vez crear un tributo llamado "número de días con lluvia", para colectar en el número de días de cada semana con precipitación mayor a 1 mm.  Siendo que el rango de fechas está desde 01-01_2021 a 31-12-2024

Vamos a resolverlo paso a paso.
Supongamos que tienes un **DataFrame `df`** con una columna de fechas (`'fecha'`) y otra con precipitación (`'precipitacion'`), con frecuencia **diaria** desde el **01-01-2021 al 31-12-2024**.

El objetivo es:

* Remuestrear de **diario a semanal**, con semanas **epidemiológicas (domingo a sábado)**.
* Crear un atributo `"dias_con_lluvia"` que cuente cuántos días en cada semana tuvieron precipitación **> 1 mm**.

---



# Clave técnica

En `pandas`, el parámetro de frecuencia `'W-SAT'` indica **semanas que terminan el sábado**, es decir, **domingo a sábado**, que coincide con las semanas epidemiológicas.

---



### ✅ Ejemplo completo en Python



In [3]:
import pandas as pd
import numpy as np

# Crear rango de fechas diarias
fechas = pd.date_range(start="2021-01-01", end="2024-12-31", freq="D")

# Simular precipitación diaria (puedes reemplazarla por tus datos reales)
np.random.seed(42)
precipitacion = np.random.gamma(shape=2, scale=2, size=len(fechas))  # valores aleatorios

# Crear DataFrame
df = pd.DataFrame({
    "fecha": fechas,
    "precipitacion": precipitacion
})

# Asegurar que la columna fecha sea índice tipo datetime
df["fecha"] = pd.to_datetime(df["fecha"])
df = df.set_index("fecha")

# Definir función personalizada para contar días con lluvia > 1 mm
def dias_con_lluvia(x):
    return (x > 1).sum()

# Remuestrear a frecuencia semanal (domingo a sábado)
df_semana = df.resample("W-SAT").agg({
    "precipitacion": ["sum", dias_con_lluvia]
})

# Renombrar columnas
df_semana.columns = ["precipitacion_total", "dias_con_lluvia"]

# Mostrar primeras filas
df_semana.head()


,precipitacion_total,dias_con_lluvia
fecha,,
2021-01-02,7.776288,2
2021-01-09,31.761719,7
2021-01-16,23.244774,6
2021-01-23,17.181001,6
2021-01-30,28.104019,7


```

---

### 🔍 Explicación

* `resample('W-SAT')` → Agrupa las fechas por semana epidemiológica (domingo–sábado).
* `.agg({...})` → Aplica múltiples funciones de agregación a la columna `'precipitacion'`.
* La función `dias_con_lluvia(x)` cuenta cuántos días en esa semana superan 1 mm de precipitación.

---

### 📊 Resultado (ejemplo)

| fecha      | precipitación_media | precipitación_total | días_con_lluvia |
| ---------- | ------------------- | ------------------- | --------------- |
| 2021-01-02 | 3.15                | 22.05               | 6               |
| 2021-01-09 | 2.84                | 19.88               | 5               |
| 2021-01-16 | 1.47                | 10.29               | 4               |
| ...        | ...                 | ...                 | ...             |

---

¿Quieres que además te añada una columna con el **número de la semana epidemiológica (1 a 52/53)** y el **año epidemiológico** (útil para análisis epidemiológicos o de salud pública)?
